In [2]:
import kss
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [5]:
def standard_text_to_dialect_text(prompt):
    with torch.no_grad():
        splited_sentences = kss.split_sentences(prompt)
        generated = []
        for sentence in splited_sentences:
            tokens = tokenizer.encode(sentence, return_tensors="pt").to(
                device="cuda", non_blocking=True
            )
            gen_tokens = model.generate(
                tokens, max_length=256, repetition_penalty=1.5
            )
            generated.append(
                tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]
            )
    return ' '.join(generated)

In [6]:
T2T_MODEL_NAME = "dannykm/DialectKoUL2"

tokenizer = AutoTokenizer.from_pretrained(T2T_MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(T2T_MODEL_NAME).to(device="cuda")
_ = model.eval()

prompt = input("텍스트를 입력하세요: ")
dialect_text = standard_text_to_dialect_text(prompt)
print(f"사투리로 변환된 텍스트: {dialect_text}")

# 이후로 wav 파일 나오게 해주시면 감사하겠습니다!

텍스트를 입력하세요: 밥 많이 먹었니?
사투리로 변환된 텍스트: 밥 많이 먹었나?
